In [4]:
import yaml

with open('designs.yaml', 'r') as file:
    experiment_definitions = yaml.safe_load(file)
print('export NUMDESIGNS=200;')
print('export TEMPLATE=/opt/xchem-fragalysis-2/mferla/dengue/mirrored.pdb;')
print('export OUTPUT_FOLDER=/opt/xchem-fragalysis-2/mferla/dengue/rf_output;')
print('cd /opt/xchem-fragalysis-2/mferla/dengue')
for definition in experiment_definitions.values():
    definition['hotspots'] = definition.get('hotspots', '')
    print('export EXPERIMENT={codename}; export HOTSPOTS=\'{hotspots}\';export CONTIGMAP=\'{contig}\'; sbatch rf.slurm.sh'.format(**definition))

export NUMDESIGNS=200;
export TEMPLATE=/opt/xchem-fragalysis-2/mferla/dengue/mirrored.pdb;
export OUTPUT_FOLDER=/opt/xchem-fragalysis-2/mferla/dengue/rf_output;
cd /opt/xchem-fragalysis-2/mferla/dengue
export EXPERIMENT=mirrored; export HOTSPOTS='';export CONTIGMAP='[A1-40/10-20/A41-196/0]'; sbatch rf.slurm.sh


    source $CONDA_PREFIX/etc/profile.d/conda.sh
    
    conda activate RFdiffusion2;
    
    
    export OUTPUT_FOLDER=/opt/xchem-fragalysis-2/mferla/dengue/rf_output;
    mkdir $OUTPUT_FOLDER
    rm STOP
    export TEMPLATE=/opt/xchem-fragalysis-2/mferla/dengue/mirrored2.pdb;
    for i in {15..40}; do
        if [[ -f STOP ]]; then
            echo "STOP file detected. Exiting loop."
            break
        fi
        export CONTIGMAP='[A1-40/'$i'/A41-196/0 A1001-1040/'$i'/A1041-1196/0]';
        export NUMDESIGNS=50;
        export HOTSPOTS='[A42,A66,A68,A88,A98,A99,A105,A113,A144,A194,A1042,A1066,A1068,A1088,A1098,A1099,A1105,A1113,A1144,A1194]'
        export EXPERIMENT=mirroredB${i}
        run_inference.py \
        --config-path=$RFDIFFUSION_CONFIG \
        hydra.output_subdir=$OUTPUT_FOLDER \
        inference.output_prefix=$OUTPUT_FOLDER/$EXPERIMENT \
        inference.input_pdb=$TEMPLATE \
        inference.write_trajectory=false \
        inference.symmetry="C2" \
        contigmap.contigs="$CONTIGMAP" \
        inference.num_designs=$NUMDESIGNS \
        'potentials.guiding_potentials=["type:olig_contacts,weight_intra:1,weight_inter:0.06"]' potentials.olig_intra_all=True potentials.olig_inter_all=True potentials.guide_scale=2 potentials.guide_decay="quadratic" \
        ;
    done;



In [ ]:
#     source $CONDA_PREFIX/etc/profile.d/conda.sh
    
#     conda activate RFdiffusion2;
    
    
#     export OUTPUT_FOLDER=/opt/xchem-fragalysis-2/mferla/dengue/rf_output;
#     mkdir $OUTPUT_FOLDER
#     rm STOP
#     export TEMPLATE=/opt/xchem-fragalysis-2/mferla/dengue/mirrored.pdb;
    
#         export CONTIGMAP='[A1-40/'$i'/A41-196/0 B1-196/0]';
#         export NUMDESIGNS=200;
#         export HOTSPOTS='[A42,A66,A68,A88,A98,A99,A105,A113,A144,A194]'
#         export EXPERIMENT=half
#         run_inference.py \
#         --config-path=$RFDIFFUSION_CONFIG \
#         hydra.output_subdir=$OUTPUT_FOLDER \
#         inference.output_prefix=$OUTPUT_FOLDER/$EXPERIMENT \
#         inference.input_pdb=$TEMPLATE \
#         inference.write_trajectory=false \
#         contigmap.contigs="$CONTIGMAP" \
#         inference.num_designs=$NUMDESIGNS \
#         ppi.hotspot_res="$HOTSPOTS" \
#         ;
#     done;


In [7]:
','.join(map('B{}'.format, sorted([144, 98, 99, 194, 113, 88, 105, 68, 66, 42])))

'B42,B66,B68,B88,B98,B99,B105,B113,B144,B194'

In [132]:
len(d[name]['A']), len(d[name]['B'])

(196, 196)

    source $CONDA_PREFIX/etc/profile.d/conda.sh
    
    conda activate RFdiffusion2;
    
    export PROTEINMPNN_WEIGHTS=$DATA/mferla/.cache/ProteinMPNN_weights/soluble_model_weights
    export WORKPATH='rf_output'
    export SERIALNUMBER='_mod'
    python $CONDA_PREFIX/bin/protein_mpnn_run.py \
            --jsonl_path "$WORKPATH/chains_definitions$SERIALNUMBER.jsonl" \
            --chain_id_jsonl "$WORKPATH/fixed_chains$SERIALNUMBER.json" \
            --fixed_positions_jsonl "$WORKPATH/fixed_positions$SERIALNUMBER.json" \
            --tied_positions_jsonl "$WORKPATH/tied_positions$SERIALNUMBER.json" \
            --out_folder $WORKPATH \
            --num_seq_per_target 5 \
            --sampling_temp "0.1" \
            --seed 888 \
            --batch_size 1 \
            --path_to_model_weights $PROTEINMPNN_WEIGHTS

In [143]:
from pathlib import Path
import re
import pandas as pd
from typing import List

def parse_header(header):
    return dict(re.findall(r'([\w_]+)=([^\s,]+)', header))

def parse_seq_dict(seq_folder: Path) -> List[dict]:
    seq_info = []
    for path in Path(seq_folder).glob('*.fa'):
        lines = path.read_text().strip().split('\n')
        if len(lines) < 2:
            print(f'{path} is empty...')
            continue  # 'not my problem'
        first = {'model': lines[0][1:].split(',')[0], 'variant': 0,**parse_header(lines[0]), 'is_control': True, 'seq':  lines[1]}
        seq_info.append(first)
        for i, (header, seq) in enumerate(zip(lines[2::2], lines[3::2])):
            current = {**first, 'variant': i+1, 'is_control': False, **parse_header(header), 'seq': seq}
            seq_info.append(current)
    return seq_info

def seqs2df(seq_folder):
    seq_info: List[dict] = parse_seq_dict(seq_folder)
    df = pd.DataFrame(seq_info)
    df['score'] = df['score'].astype(float)
    df['global_score'] = df['global_score'].astype(float)
    df['seq_recovery'] = df['seq_recovery'].astype(float)
    df['letter'] = df.variant.map({0: 'Ø', 1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'F'})
    return df.sort_values('global_score').copy()

In [159]:
df = seqs2df(seq_folder=Path('rf_output/seqs'))
print(len(df))
df

rf_output/seqs/mirrored28_2.fa is empty...
132


,model,variant,score,global_score,fixed_chains,designed_chains,model_name,git_hash,seed,is_control,seq,T,sample,seq_recovery,letter
51,mirrored39_7,3,0.9950,1.2649,[],['A',v_48_020,unknown,888,False,MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKEPVEVE...,0.1,3,0.0513,C
52,mirrored39_7,4,1.0122,1.2839,[],['A',v_48_020,unknown,888,False,MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKEDVSVT...,0.1,4,0.0513,D
50,mirrored39_7,2,0.9874,1.2851,[],['A',v_48_020,unknown,888,False,MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKEPVSVT...,0.1,2,0.0513,B
49,mirrored39_7,1,0.9773,1.2851,[],['A',v_48_020,unknown,888,False,MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKTDVSVT...,0.1,1,0.0513,A
33,mirrored37_6,3,1.1413,1.2933,[],['A',v_48_020,unknown,888,False,MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKTPTTVE...,0.1,3,0.0000,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,mirrored37_5,0,3.6400,1.7377,[],['A',v_48_020,unknown,888,True,MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKGGGGGG...,NaN,NaN,NaN,Ø
42,mirrored36_4,0,3.6684,1.7397,[],['A',v_48_020,unknown,888,True,MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKGGGGGG...,NaN,NaN,NaN,Ø
54,mirrored34_9,0,3.6201,1.7401,[],['A',v_48_020,unknown,888,True,MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKGGGGGG...,NaN,NaN,NaN,Ø
120,mirrored33_4,0,3.7132,1.7412,[],['A',v_48_020,unknown,888,True,MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKGGGGGG...,NaN,NaN,NaN,Ø


In [50]:
import numpy as np



[(np.arange(1, plddt.shape[0])[idx] + 1, plddt[idx]) for idx in np.where(plddt < 0.8)[0]]

[(46, 0.777318),
 (47, 0.71042556),
 (48, 0.6941745),
 (49, 0.6564318),
 (50, 0.6540519),
 (51, 0.57856476),
 (52, 0.66202664),
 (53, 0.7174145),
 (54, 0.7847134),
 (257, 0.7800815),
 (258, 0.7132509),
 (259, 0.6952838),
 (260, 0.6573798),
 (261, 0.65568507),
 (262, 0.5751328),
 (263, 0.6606506),
 (264, 0.7162048),
 (265, 0.7840113)]

In [53]:
'+'.join(map(str, [np.arange(1, plddt.shape[0])[idx] + 1 for idx in np.where(plddt < 0.8)[0]]))

'46+47+48+49+50+51+52+53+54+257+258+259+260+261+262+263+264+265'

In [ ]:
MADLELERAADVKWEDQAEISGSSPILSITISEDGSMSIKAELEDGAYRIKQKGILGYSQIGAGVYKEGT
FHTMWHVTRGAVLMHKGKRIEPSWADVKKDLISYGGGWKLEGEWKEGEEVQVLALEPGKNPRAVQTKPGL
FKTNAGTIGAVSLDFSPGTSGSPIIDKKGKVVGLYGNGVVTRSGAYVSAIAQTEKS

32/3.8 = 8.5